Este notebook es para generar el paso de tener parrafos o un texto segmentado y generar codes para estos o un hasthag, despues de generar el hasthag vamos a vectorizarlos en altas dimensiones y aplicaremos HBDSCAN ahi mismo en las altas dimensiones, despues ya con los clusters vamos a pedir al LLM un theme para el cluster, para darles nombre y al final los visualizamos


In [ ]:
pip install sentence-transformers umap-learn scikit-learn matplotlib seaborn pandas numpy optuna ollama

In [ ]:
import pandas as pd


transcripts_Armando = "https://raw.githubusercontent.com/3eltran23/ATA-LMM_study/refs/heads/main/data/transcripts.csv"
transcripts_Armando = pd.read_csv(transcripts_Armando)

codes_Armando = "https://raw.githubusercontent.com/3eltran23/ATA-LMM_study/refs/heads/main/data/combined_codes.csv"
codes_Armando = pd.read_csv(codes_Armando)

Hockey_Interviews = "https://raw.githubusercontent.com/HiramZ04/Cal-State-UMAP_Clustering/refs/heads/main/data/National-Hockey-League-Interviews.csv"
Hockey_Interviews = pd.read_csv(Hockey_Interviews)

callrecordings = "https://raw.githubusercontent.com/HiramZ04/Cal-State-UMAP_Clustering/refs/heads/main/data/call_recordings.csv"
callrecordings = pd.read_csv(callrecordings)

transcripts_Armando.head(5)

,Filename,Index,Segment/Excerpt
0,p008-parent,1,Parent:\tI think you had asked me what my conc...
1,p008-parent,2,for Reed Richards having it.
2,p008-parent,3,"So, him losing it, it kept making damage somehow."
3,p008-parent,4,Those were really my biggest concerns.\nInterv...
4,p008-parent,5,Do you or your son?\nParent:\tThe three of us ...


In [22]:
codes_Armando.head(5)

,Codes,Source
0,Reinforcing school dynamic at home,p008-parent
1,Functionalities of smartwatch,p008-parent
2,Daily or weekly goals,p008-parent
3,Sharing information with separated parents,p008-parent
4,Pick up of watch and phone,p008-parent


In [27]:
Hockey_Interviews.head(5)

,RowId,team1,team2,date,name,job,text
0,0,blackhawks,lightning,2015-06-02,stan bowman,other,well we're very fortunate to have the players ...
1,1,blackhawks,lightning,2015-06-02,steve yzerman,other,no we didn't really set a timeline on it i wou...
2,2,blackhawks,lightning,2015-06-03,antoine vermette,player,that's a good question i don't recall specific...
3,3,blackhawks,lightning,2015-06-03,joel quenneville,coach,yeah we got better as the game went on i thoug...
4,4,blackhawks,lightning,2015-06-03,jon cooper,coach,i don't know i think the way i'd look at the g...


In [36]:
callrecordings.head(5)

,id,Type,Sentiment,Name,Order Number,Product Number,Transcript
0,call_recording_01,Product Inquiry,Neutral,Sarah Miller,NaN,AC-7892,"Hello, I'm Sarah Miller. I'm calling to inquir..."
1,call_recording_02,Complaint,Angry,John Davis,123456.0,FR-4401,I am extremely dissatisfied with my recent ord...
2,call_recording_03,Technical Issue,Frustrated,Maria Rodriguez,987654.0,LAP-2110,"Hi, this is Maria Rodriguez. I'm having troubl..."
3,call_recording_04,Compliment,Happy,Robert Smith,246801.0,DW-6543,I just wanted to call and say how pleased I am...
4,call_recording_05,Order Placement,Neutral,Jessica Brown,NaN,OV-1357 & MW-8642,"Hello, my name is Jessica Brown. I'd like to p..."


In [63]:
from ollama import chat
# We start passing a transcript to LLM
# We use gemma2:9b  

prompt_template = """
    You are a qualitative research analyst. Read one interview segment and produce EXACTLY ONE thematic label.

Requirements for the label:
- 2–6 words, concise and descriptive
- Neutral, observable phrasing (no opinions or causes)
- Title Case, no punctuation, no quotes
- Include at least one salient noun from the segment (e.g., product name, parameter, entity)


    Output Format:
    - ONLY 1 CODE and nothing else

    """

In [ ]:
# le pasamos las palabras al llm 
# CALL CENTER DATA SET

sentences = callrecordings["Transcript"].astype(str).tolist()
    
codes = []
GEN_OPTS = {"temperature": 0.0, "top_p": 1.0, "repeat_penalty": 1.2}

for i in range(len(sentences)): 
# Pasamos el prompt con un system para que nos regrese el output que deseamos
    messages = [
        {
        'role':'system','content':prompt_template
        },
        {
        'role':'user', 'content':f'This is the transcript i need the codes for: {i}',
        },   
    ]

    label = chat(model = "gemma2:9b", messages=messages, options=GEN_OPTS)
    codes.append(label['message']['content'])
    print("LLM response: ",label['message']['content'])

print("Lista completa de codes: ",codes)


LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

LLM response:  Product Inquiry  

Lista completa de codes:  ['Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry  \n', 'Product Inquiry 

In [53]:
# HOCKEY INTERVIEWS DATASET

sentences = Hockey_Interviews["text"]
    
codes = []

for i in range(len(sentences)): 
# Pasamos el prompt con un system para que nos regrese el output que deseamos
    messages = [
        {
        'role':'system','content':prompt_template
        },
        {
        'role':'user', 'content':f'This is the transcript i need the codes for: {sentences}',
        },   
    ]

    label = chat(model = "gemma2:9b", messages=messages)
    codes.append(label['message']['content'])
    print("LLM response: ",label['message']['content'])

print("Lista completa de codes: ",codes)


LLM response:  Team Performance  

LLM response:  Game Analysis  

LLM response:  Game Analysis  

LLM response:  Team Performance  

LLM response:  Team Performance Analysis  

LLM response:  Player Performance and Fatigue  

LLM response:  Game Performance and Reflection  

LLM response:  Team Performance  

LLM response:  Post-game analysis  

LLM response:  Post-game analysis  

LLM response:  Game Performance Analysis  

LLM response:  Team Performance  

LLM response:  Post-Game Analysis  

LLM response:  Team Performance Analysis  

LLM response:  Team Performance Analysis  

LLM response:  Player Performance  

LLM response:  Post-game Analysis  

LLM response:  Team Performance Analysis  

LLM response:  Post-game Analysis 

LLM response:  Player Performance Analysis  

LLM response:  Post-game analysis 

LLM response:  Post-Game Analysis  

LLM response:  Post-game analysis  

LLM response:  Team Performance and Analysis  

LLM response:  Game Analysis  

LLM response:  Team 

KeyboardInterrupt: 